In [1]:
# hit API 
import requests

# parsing data
import pandas as pd
from datetime import date

# scraping progress
from tqdm import tqdm

# connecting to db
import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("DANMAR LINES")

In [3]:
# key to map key from json request to key used in db
Mapping = { "Estimated Vessel Departure"  : "ETD",
            "Actual Vessel Departure" : "ATD",
            "Estimated Vessel Arrival" : "ETA",
            "Actual Vessel Arrival" : "ATA"}

In [4]:
bl_list

['BDOA02975',
 'BDOA02990',
 'BDOA02980',
 'BDOA02949',
 'BDOA02934',
 'BDOA02977',
 'BDOA02988',
 'BDOA02960',
 'BDOA02979',
 'BDOA02952',
 'BDOA02978',
 'BDOA02989',
 'BDOA02968',
 'BDOA02961']

In [5]:
list_of_dict = []

for i, bls in enumerate(tqdm(bl_list)):
    try:
    
        print("BL in Process: " ,bls)
        
        # hitting API
        url = "https://api.dhl.com/dgff/transportation/shipment-tracking"
        querystring = {"housebill": bls}
        headers = {
            "content-type": "application/json",
            "DHL-API-Key": "qEHPTpIAc13hHfjB6H6gJQ0vEfmP6GhA"
        }

        # pasting json response
        response = requests.request("GET", url, headers=headers, params=querystring)
        json_data = response.json() if response and response.status_code == 200 else None

        temp_dict={}
        
        # list of containers in 1 bl
        list_of_containers=[]
        for i in json_data['ShipmentTracking']['Shipment']['TransportUnits']['TransportUnit']:
            list_of_containers.append(i['TransportUnitID'])

        print(list_of_containers)

        # parsing every data per container
        for c, container in enumerate(tqdm(list_of_containers)):
            current_dict={}
            current_dict.update({"Liners" : "Danmar Lines / DHL"})
            current_dict.update({"BL Number" : bls})
            current_dict.update({"Container Number" : container})
            current_dict.update({"From" : json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'][:json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'].index(",")]})
            current_dict.update({"To" : json_data['ShipmentTracking']['Shipment']['Destination']['LocationName']})
            
            for j, status_dict in enumerate (json_data['ShipmentTracking']['Shipment']['Timestamps']['Timestamp']):
                status_name = status_dict.get('TimestampDescription')
                status_date = status_dict.get('TimestampDateTime')
                case_data = {status_name:status_date[:10]}
                temp_dict.update(case_data)
                
            for old_key, new_key in Mapping.items():
                try:
                    current_dict[new_key] = temp_dict[old_key]
                except Exception as e:
                    current_dict[new_key] = None
                    print("THIS IS ERROR====>",e)

            list_of_dict.append(current_dict)
            print(bls, container,"done")
    except Exception as e:
        print(bls, "THIS IS ERROR====>",e)

  0%|          | 0/14 [00:00<?, ?it/s]

BL in Process:  BDOA02975
['KMTU9350991', 'BMOU6243329', 'FCIU7542981', 'TXGU5033010', 'SEGU5574968']


  7%|▋         | 1/14 [00:01<00:14,  1.13s/it]

BDOA02975 KMTU9350991 done
BDOA02975 BMOU6243329 done
BDOA02975 FCIU7542981 done
BDOA02975 TXGU5033010 done
BDOA02975 SEGU5574968 done
BL in Process:  BDOA02990
['BMOU6043146']


 14%|█▍        | 2/14 [00:01<00:10,  1.11it/s]

...

BDOA02989 BEAU6191534 done
BL in Process:  BDOA02968
['TXGU6144446', 'SEGU4121421', 'KMTU9342580', 'FCIU7070490', 'TEMU7819794']


 93%|█████████▎| 13/14 [00:10<00:00,  1.27it/s]

BDOA02968 TXGU6144446 done
BDOA02968 SEGU4121421 done
BDOA02968 KMTU9342580 done
BDOA02968 FCIU7070490 done
BDOA02968 TEMU7819794 done
BL in Process:  BDOA02961
['OOCU8685761']


100%|██████████| 14/14 [00:11<00:00,  1.22it/s]

BDOA02961 OOCU8685761 done


In [6]:
list_of_dict[-1]

{'Liners': 'Danmar Lines / DHL',
 'BL Number': 'BDOA02961',
 'Container Number': 'OOCU8685761',
 'From': 'Jakarta',
 'To': 'Port Klang',
 'ETD': '2023-09-14',
 'ATD': '2023-09-13',
 'ETA': '2023-09-22',
 'ATA': '2023-09-22'}

In [7]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict)

Today's Collection Name ===>  all_tracking_Oct-06-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [20]:
# exporting scraped data to excel
df = pd.DataFrame(list_of_dict)
excel_file_path = 'export excel/DHL.xlsx'
df.to_excel(excel_file_path, index=False)